In [47]:
total_value=int(input("Enter the total money(in Numerical form)"))

Enter the total money(in Numerical form)10000000


In [30]:
Num_Companies=int(input("Enter the total money(in Numerical form)"))

Enter the total money(in Numerical form)40


In [25]:
import datapackage
import pandas as pd

data_url = 'https://datahub.io/core/s-and-p-500-companies/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        dataset = pd.read_csv(resource.descriptor['path'])
sym_name=dict()
for i in range(len(dataset["Symbol"])):
    sym_name[dataset["Symbol"][i]]=dataset["Name"][i]
dataset

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [26]:
def splitter(symbols,size):
    li=list()
    start=0
    while start<len(symbols):
        x=""
        if start+size<len(symbols):
            end=start+size
        else:
            end=len(symbols)
        for i in range(start,end):
            x+=(symbols[i]+",")
        x=x[:-1]
        li.append(x)
        start+=size
    return li

In [27]:
import numpy as np
import requests
import time 
#please enter your generated token key in token platform 
token=""
symbols=dataset["Symbol"]
splits=splitter(symbols,80)

In [28]:
splits=splitter(symbols,80)
li=list()
for sym in splits:
    time.sleep(2)
    #in case you are using actual version instead of sandbox please remove sandbox from url
    url='https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols='+sym+'&token='+token
    data=requests.get(url).json()
    for i in data:
        name=sym_name[i]
        li.append([name,i,data[i]['quote']['latestPrice'],
               'N/A',
               data[i]['stats']['year1ChangePercent'],
               'N/A',
               data[i]['stats']['month6ChangePercent'],
               'N/A',
               data[i]['stats']['month3ChangePercent'],
               'N/A',
               data[i]['stats']['month1ChangePercent'],
               'N/A',
               'N/A'])
stockset=pd.DataFrame(li,columns=["Company",'Symbol', 'Price', 'Quantity',
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'])
percompanyvalue=total_value/len(stockset["Symbol"])
  

In [29]:
stockset

,Company,Symbol,Price,Quantity,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,3M,MMM,167.00,N/A,-0.043925,N/A,-0.167548,N/A,-0.080128,N/A,-0.088942,N/A,N/A
1,A. O. Smith,AOS,75.60,N/A,0.416854,N/A,0.082943,N/A,0.037141,N/A,-0.120103,N/A,N/A
2,Abbott Laboratories,ABT,125.74,N/A,0.030545,N/A,0.043200,N/A,-0.024845,N/A,-0.113087,N/A,N/A
3,AbbVie,ABBV,142.11,N/A,0.416706,N/A,0.193876,N/A,0.219872,N/A,0.029889,N/A,N/A
4,Abiomed,ABMD,281.28,N/A,-0.207498,N/A,-0.151248,N/A,-0.168164,N/A,-0.237690,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Yum! Brands,YUM,126.08,N/A,0.225512,N/A,-0.056274,N/A,-0.017978,N/A,-0.119718,N/A,N/A
500,Zebra Technologies,ZBRA,516.31,N/A,0.270219,N/A,-0.104904,N/A,-0.079637,N/A,-0.183739,N/A,N/A
501,Zimmer Biomet,ZBH,124.42,N/A,-0.220177,N/A,-0.261449,N/A,-0.157801,N/A,-0.059383,N/A,N/A
502,Zions Bancorp,ZION,71.42,N/A,0.589677,N/A,0.316432,N/A,0.091232,N/A,0.074806,N/A,N/A


In [36]:
from scipy import stats
from statistics import mean
for row in stockset.index:
    stockset.loc[row,'One-Year Return Percentile'] = stats.percentileofscore(stockset['One-Year Price Return'], stockset.loc[row, 'One-Year Price Return'])
    stockset.loc[row,'Six-Month Return Percentile'] = stats.percentileofscore(stockset['Six-Month Price Return'], stockset.loc[row, 'Six-Month Price Return'])
    stockset.loc[row,'Three-Month Return Percentile'] = stats.percentileofscore(stockset['Three-Month Price Return'], stockset.loc[row, 'Three-Month Price Return'])
    stockset.loc[row,'One-Month Return Percentile'] = stats.percentileofscore(stockset['One-Month Price Return'], stockset.loc[row, 'One-Month Price Return'])
    a=[stockset.loc[row,'One-Year Return Percentile'],stockset.loc[row,'Six-Month Return Percentile'],stockset.loc[row,'Three-Month Return Percentile'],stockset.loc[row,'One-Month Return Percentile']]
    stockset.loc[row,'HQM Score']=mean(a)
stockset

,Company,Symbol,Price,Quantity,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,3M,MMM,167.00,N/A,-0.043925,13.2937,-0.167548,13.0952,-0.080128,30.5556,-0.088942,39.4841,24.1071
1,A. O. Smith,AOS,75.60,N/A,0.416854,79.3651,0.082943,73.0159,0.037141,68.4524,-0.120103,29.3651,62.5496
2,Abbott Laboratories,ABT,125.74,N/A,0.030545,22.2222,0.043200,62.5,-0.024845,48.6111,-0.113087,30.754,41.0218
3,AbbVie,ABBV,142.11,N/A,0.416706,79.1667,0.193876,88.8889,0.219872,97.2222,0.029889,82.9365,87.0536
4,Abiomed,ABMD,281.28,N/A,-0.207498,3.96825,-0.151248,15.873,-0.168164,11.3095,-0.237690,3.76984,8.73016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Yum! Brands,YUM,126.08,N/A,0.225512,54.1667,-0.056274,35.7143,-0.017978,50.7937,-0.119718,29.7619,42.6091
500,Zebra Technologies,ZBRA,516.31,N/A,0.270219,61.3095,-0.104904,23.0159,-0.079637,31.1508,-0.183739,11.1111,31.6468
501,Zimmer Biomet,ZBH,124.42,N/A,-0.220177,3.1746,-0.261449,4.7619,-0.157801,14.4841,-0.059383,50.3968,18.2044
502,Zions Bancorp,ZION,71.42,N/A,0.589677,92.6587,0.316432,96.627,0.091232,81.3492,0.074806,91.4683,90.5258


In [42]:
stockset.sort_values(by = 'HQM Score', ascending = False,inplace=True)
stockset=stockset[:Num_Companies]
stockset.reset_index(drop=True,inplace=True)

In [43]:
stockset

,Company,Symbol,Price,Quantity,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,Devon Energy,DVN,51.56,N/A,2.385482,100,1.000915,99.8016,0.318375,99.8016,0.166903,97.619,99.3056
1,Marathon Oil,MRO,20.27,N/A,1.829064,99.8016,0.693301,99.4048,0.224986,97.619,0.204175,98.6111,98.8591
2,EOG Resources,EOG,110.90,N/A,1.301889,99.2063,0.551286,98.8095,0.223268,97.4206,0.233265,99.2063,98.6607
3,Halliburton,HAL,32.43,N/A,0.811745,97.2222,0.491195,98.2143,0.261343,98.8095,0.370516,100,98.5615
4,Diamondback Energy,FANG,128.91,N/A,1.345995,99.6032,0.661007,99.2063,0.206710,96.627,0.184806,98.2143,98.4127
5,ConocoPhillips,COP,90.84,N/A,1.319256,99.4048,0.595374,99.0079,0.202776,96.4286,0.227240,98.8095,98.4127
6,APA Corporation,APA,33.31,N/A,0.725729,95.4365,0.769504,99.6032,0.282502,99.6032,0.195466,98.4127,98.2639
7,Schlumberger,SLB,41.45,N/A,0.841141,97.4206,0.359609,97.2222,0.239419,98.2143,0.351801,99.8016,98.1647
8,ExxonMobil,XOM,78.07,N/A,0.808080,97.0238,0.318811,96.8254,0.187133,95.0397,0.234372,99.4048,97.0734
9,Pioneer Natural Resources,PXD,220.90,N/A,0.874047,97.8175,0.521327,98.4127,0.175505,94.0476,0.177761,97.8175,97.0238


In [48]:
for i in range(len(stockset["Quantity"])):
    stockset['Quantity'][i]=total_value//(Num_Companies*stockset["Price"][i])
stockset

<ipython-input-48-ae39e912f88e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stockset['Quantity'][i]=total_value//(Num_Companies*stockset["Price"][i])
C:\Users\Akshay\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Company,Symbol,Price,Quantity,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,Devon Energy,DVN,51.56,4848,2.385482,100,1.000915,99.8016,0.318375,99.8016,0.166903,97.619,99.3056
1,Marathon Oil,MRO,20.27,12333,1.829064,99.8016,0.693301,99.4048,0.224986,97.619,0.204175,98.6111,98.8591
2,EOG Resources,EOG,110.90,2254,1.301889,99.2063,0.551286,98.8095,0.223268,97.4206,0.233265,99.2063,98.6607
3,Halliburton,HAL,32.43,7708,0.811745,97.2222,0.491195,98.2143,0.261343,98.8095,0.370516,100,98.5615
4,Diamondback Energy,FANG,128.91,1939,1.345995,99.6032,0.661007,99.2063,0.206710,96.627,0.184806,98.2143,98.4127
5,ConocoPhillips,COP,90.84,2752,1.319256,99.4048,0.595374,99.0079,0.202776,96.4286,0.227240,98.8095,98.4127
6,APA Corporation,APA,33.31,7505,0.725729,95.4365,0.769504,99.6032,0.282502,99.6032,0.195466,98.4127,98.2639
7,Schlumberger,SLB,41.45,6031,0.841141,97.4206,0.359609,97.2222,0.239419,98.2143,0.351801,99.8016,98.1647
8,ExxonMobil,XOM,78.07,3202,0.808080,97.0238,0.318811,96.8254,0.187133,95.0397,0.234372,99.4048,97.0734
9,Pioneer Natural Resources,PXD,220.90,1131,0.874047,97.8175,0.521327,98.4127,0.175505,94.0476,0.177761,97.8175,97.0238


In [62]:
import xlsxwriter
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
stockset.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [63]:
background_color = '#ffffff'
font_color = '#000000'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [64]:
column_formats = { 
                    'A': ['Company',string_template],
                    'B': ['Symbol', string_template],
                    'C': ['Price', dollar_template],
                    'D': ['Quantity', integer_template],
                    'E': ['One-Year Price Return', percent_template],
                    'F': ['One-Year Return Percentile', percent_template],
                    'G': ['Six-Month Price Return', percent_template],
                    'H': ['Six-Month Return Percentile', percent_template],
                    'I': ['Three-Month Price Return', percent_template],
                    'J': ['Three-Month Return Percentile', percent_template],
                    'K': ['One-Month Price Return', percent_template],
                    'L': ['One-Month Return Percentile', percent_template],
                    'M': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [65]:
writer.save()